In [1]:
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

ImportError: cannot import name 'ENGLISH_CONNECTOR_WORDS'

In [10]:
import gensim

In [5]:
!pip install --user gensim --upgrade

     |████████████████████████████████| 23.9MB 11.0MB/s eta 0:00:01     |████████████████████▉           | 15.6MB 11.0MB/s eta 0:00:01     |███████████████████████▉        | 17.8MB 11.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/e9/90/6ca525991e281ecdf204c5c1de854da6334068e44121c384b68c6a838e14/smart_open-5.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fe/ca/75fac5856ab5cfa51bbbcefa250182e50441074fdc3f803f6e76451fab43/dataclasses-0.8-py3-none-any.whl


In [6]:
# Each sentence must be a list of string tokens:

In [11]:
print(gensim.__version__)

3.7.3


In [11]:
from afinn import Afinn
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import sentiwordnet as swn

In [ ]:
#import nltk
#nltk.download('vader_lexicon')


In [32]:
def afinn_sentiment(text):
    return Afinn().score(text)

def textblob_polarity(text):
    return TextBlob(text).sentiment.polarity

def vader_sentiment(text):
    return SentimentIntensityAnalyzer().polarity_scores(text)

In [ ]:
# template I think 

def sentiment_score():
    
    df['afinn'] = df[df['text']].apply(afinn_score)
    
    df['textblob'] = df[df['text']].apply(textblob_polarity)
    
    df['vader'] = df[df['text']].apply(vader_sentiment)
    df['vader'] = df[df['vader']].apply(lambda score_dict: score_dict['compound'])
    df['vader'] = df[df['vader']].apply(lambda c: 'pos' if c >=0 else 'neg')
    
    return out 